In [54]:
import dash
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import plotly.graph_objs as go

# Load your data
file_path = 'data/weekly_avg_us'  # Replace with your file path
df = pd.read_csv(file_path)

# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])
df['date_num'] = (df['date'] - df['date'].min()).dt.days

###########################################################

state_data = pd.read_csv('data/4states.csv')  # Replace with your state data file path
state_data['date'] = pd.to_datetime(state_data['date'])




us_population = 336997632

# Build App
app = Dash(__name__)
app.layout = html.Div([
    html.H1("COVID-19 Data Visualization", style={'text-align': 'center'}),
    html.Div([
        dcc.Graph(id='graph'),
        html.Div([
            
            html.Label("Select Category:"),
            dcc.RadioItems(
                id='data-category',
                options=[
                    {'label': 'Total Cases', 'value': 'total_cases'},
                    {'label': 'Total Deaths', 'value': 'total_deaths'},
                    {'label': 'New Cases', 'value': 'new_cases'},
                    {'label': 'New Deaths', 'value': 'new_deaths'}
                ],
                value='total_cases'  # Default value
            ),
            html.Div([],style={'margin-bottom': '10px'}), # Add space between components
            html.Label("Select Scale:"),
            dcc.RadioItems(
                id='scale-type',
                options=[
                    {'label': 'Linear', 'value': 'linear'},
                    {'label': 'Log', 'value': 'log'}
                ],
                value='linear'  # Default value
            )
        ], style={'margin-left': '0px', 'justify-content': 'center', 'text-align': 'left', 'display': 'flex', 'flex-direction': 'column'})
    ], style={'display': 'flex', 'justify-content': 'center', 'height': '500px', 'background-color': '#f2f2f2'}),
    html.Div([
        html.Div([ html.Label("Select Date Range:"),
            dcc.DatePickerRange(
                id='date-picker-range',
                start_date=df['date'].min(),
                end_date=df['date'].max(),
                display_format='YYYY-MM-DD'
            ),
            html.Br(),], style={'width': '20%'}),
     html.Div([
         # With this
            html.Label("Select Regression"),
            dcc.Checklist(
                id='regression-type',
                options=[
                    {'label': 'Linear Regression', 'value': 'linear'},
                    {'label': 'Polynomial Regression', 'value': 'poly'}
                ],
                value=[]  # Default value
            ), 
            html.Div([],style={'margin-bottom': '10px'}), # Add space between components
            html.Label("Select Prediction Weeks:"),
            dcc.Slider(
                id='prediction-weeks',
                min=0,
                max=5,
                step=1,
                value=0  # Default value
            )
        ], style={'margin-bottom': '20px', 'justify-content': 'center', 'text-align': 'left', 'display': 'flex', 'flex-direction': 'column'})
    ], style={'display': 'flex', 'justify-content': 'space-around', 'background-color': '#f2f2f2'}),
    
      html.H3("States Data", style={'text-align': 'center'}),
    html.Div([
        dcc.Dropdown(
            id='state-dropdown',
            options=[{'label': state, 'value': state} for state in state_data['State'].unique()],
            value=state_data['State'].unique()[0]  # Default value
        ),
        dcc.RadioItems(
            id='state-data-category',
            options=[
                {'label': 'Cases', 'value': 'cases'},
                {'label': 'Deaths', 'value': 'deaths'}
            ],
            value='cases'  # Default value
        ),
        dcc.Graph(id='state-graph')
    ], style={'display': 'flex','flex-direction':'column', 'justify-content': 'center', 'height': '500px', 'background-color': '#f2f2f2'}),
    html.Div([
        html.H2(":"),
        html.A("Emanuel Macias", href='#', target='_blank')
    ]),
   
])

@app.callback(
    Output('graph', 'figure'),
    [Input("data-category", "value"),
     Input("scale-type", "value"),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('regression-type', 'value'),
     Input('prediction-weeks', 'value')]

)
def update_figure(selected_category, scale_type, start_date, end_date, regression_type, prediction_weeks):
    # Filter data based on selected date range
    filtered_df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

    if scale_type == 'log':
        filtered_df[selected_category] = np.log10((filtered_df[selected_category] / us_population) * 100000 + 1)

    # Calculate 7-day moving average and add it to the dataframe
    filtered_df['7_day_moving_avg'] = filtered_df[selected_category].rolling(window=7).mean()

    fig = px.line(
        filtered_df, x="date", y=selected_category, title=f"Weekly Average of {selected_category}",
        log_y=(scale_type == 'log')
    )

    # Add 7-day moving average to the plot
    fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['7_day_moving_avg'],
                             mode='lines', name='7-Day Moving Average'))

    # Add regression line and predictions
    if 'linear' in regression_type:
        model = LinearRegression()
        model.fit(filtered_df['date_num'].values.reshape(-1, 1), filtered_df[selected_category])
        y_pred = model.predict(filtered_df['date_num'].values.reshape(-1, 1))
        fig.add_trace(go.Scatter(x=filtered_df['date'], y=y_pred, mode='lines', name='Linear Regression'))

        if prediction_weeks > 0:
            future_dates = np.array([filtered_df['date_num'].max() + i*7 for i in range(1, prediction_weeks+1)]).reshape(-1, 1)
            future_preds = model.predict(future_dates)
            future_dates = pd.to_datetime(df['date'].min()) + pd.to_timedelta(future_dates.flatten(), unit='D')
            fig.add_trace(go.Scatter(x=future_dates, y=future_preds, mode='lines', name='Linear Prediction'))

    if 'poly' in regression_type:
        poly = PolynomialFeatures(degree=2)
        X_poly = poly.fit_transform(filtered_df['date_num'].values.reshape(-1, 1))
        model = LinearRegression()
        model.fit(X_poly, filtered_df[selected_category])
        y_poly_pred = model.predict(X_poly)
        fig.add_trace(go.Scatter(x=filtered_df['date'], y=y_poly_pred, mode='lines', name='Polynomial Regression'))

        if prediction_weeks > 0:
            future_dates = np.array([filtered_df['date_num'].max() + i*7 for i in range(1, prediction_weeks+1)]).reshape(-1, 1)
            future_dates_poly = poly.transform(future_dates)
            future_preds = model.predict(future_dates_poly)
            future_dates = pd.to_datetime(df['date'].min()) + pd.to_timedelta(future_dates.flatten(), unit='D')
            fig.add_trace(go.Scatter(x=future_dates, y=future_preds, mode='lines', name='Polynomial Prediction'))

    return fig

@app.callback(
    Output('state-graph', 'figure'),
    [Input('state-dropdown', 'value'),
     Input('state-data-category', 'value')]
)
def update_state_graph(selected_state, data_category):
    # Filter data for the selected state
    filtered_df = state_data[state_data['State'] == selected_state]

    # Plotting the state-specific data
    fig = px.line(
        filtered_df, x='date', y=data_category, title=f'{data_category.title()} in {selected_state}'
    )

    return fig

# Run app and display result inline in the notebook
app.run_server(jupyter_mode='external')

Dash app running on http://127.0.0.1:8050/


/home/codespace/.local/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/home/codespace/.local/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

